# Segmenting and Clustering Neighborhoods in Toronto

Libraries importing

In [2]:
!pip install bs4
!pip install geopy
# Import packages
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

  Stored in directory: C:\Users\zubair.ashfaque\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Libraries imported.


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Package the request, send the request and catch the response: r
r = requests.get(url)

# Extract the response as html: html_doc
html_doc = r.text

# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html_doc)
table=soup.find('table', {'class': 'wikitable sortable'})
rows=list()
for row in table.find_all("tr"):
   rows.append(row)
l = []
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row) 
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])

In [4]:
print(df)

    PostalCode           Borough  \
0         None              None   
1          M1A      Not assigned   
2          M2A      Not assigned   
3          M3A        North York   
4          M4A        North York   
5          M5A  Downtown Toronto   
6          M5A  Downtown Toronto   
7          M6A        North York   
8          M6A        North York   
9          M7A      Queen's Park   
10         M8A      Not assigned   
11         M9A         Etobicoke   
12         M1B       Scarborough   
13         M1B       Scarborough   
14         M2B      Not assigned   
15         M3B        North York   
16         M4B         East York   
17         M4B         East York   
18         M5B  Downtown Toronto   
19         M5B  Downtown Toronto   
20         M6B        North York   
21         M7B      Not assigned   
22         M8B      Not assigned   
23         M9B         Etobicoke   
24         M9B         Etobicoke   
25         M9B         Etobicoke   
26         M9B         Etobi

Removing unwanted data

In [5]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n


In [6]:
df=df.drop_duplicates(subset='PostalCode',keep='first')
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
5,M5A,Downtown Toronto,Harbourfront\n


In [7]:
df = df.replace('\n','', regex=True)
df.loc[df['PostalCode'] == 'M7A']

,PostalCode,Borough,Neighborhood
9,M7A,Queen's Park,Not assigned


In [8]:
df.loc[df['Borough'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
10,M8A,Not assigned,Not assigned
14,M2B,Not assigned,Not assigned
21,M7B,Not assigned,Not assigned
22,M8B,Not assigned,Not assigned
31,M2C,Not assigned,Not assigned
37,M7C,Not assigned,Not assigned
38,M8C,Not assigned,Not assigned
46,M2E,Not assigned,Not assigned


In [9]:
df.loc[df['PostalCode'] == 'M7A']

,PostalCode,Borough,Neighborhood
9,M7A,Queen's Park,Not assigned


In [10]:
df = df[df.Borough != 'Not assigned']
df = df[df.Neighborhood != 'Not assigned']
df = df[df.PostalCode != 'Not assigned']

In [11]:
df = df.dropna(how='any',axis=0) 

resetting index

In [12]:
df.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M9A,Etobicoke,Islington Avenue
5,M1B,Scarborough,Rouge
6,M3B,North York,Don Mills North
7,M4B,East York,Woodbine Gardens
8,M5B,Downtown Toronto,Ryerson
9,M6B,North York,Glencairn


In [13]:
df.shape

(102, 3)

downloading CSV FILE http://cocl.us/Geospatial_data

In [14]:
path= 'https://cocl.us/Geospatial_data'
data = pd.read_csv(path, skiprows=1, names=['PostalCode', 'Latitude', 'Longitude'])
data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
neighborhoods = pd.merge(data, df, on='PostalCode')

re-arranging columns 

In [17]:
neighborhoods = neighborhoods[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [18]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1C,Scarborough,Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Checking stats of table

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


Location of Toronto

In [20]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


create map of Toronto using latitude and longitude values

In [21]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

In [22]:

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Boroughs that contain the word Toronto

In [24]:
T_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [25]:
T_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West,43.679557,-79.352188
2,M4L,East Toronto,The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,Moore Park,43.689574,-79.383160
9,M4V,Central Toronto,Deer Park,43.686412,-79.400049


create map of Boroughs word Toronto using latitude and longitude values

In [26]:

map_TOR = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(T_data['Latitude'], T_data['Longitude'], T_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TOR)  
    
map_TOR

Setting Foursquare Parameters

In [27]:
CLIENT_ID = '3HCZAEU4QTKJOPBGGQDDMNMCZZEDBBHVWKXYNZBIJX3IVCLS' # your Foursquare ID
CLIENT_SECRET = 'G1YY3I5WD1HVJCBMK5SOMKVVWZRSA4RT1ZEVEEOKBY2VC0MO' # your Foursquare Secret
VERSION = '20190925' # Foursquare API version
radius = 500
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3HCZAEU4QTKJOPBGGQDDMNMCZZEDBBHVWKXYNZBIJX3IVCLS
CLIENT_SECRET:G1YY3I5WD1HVJCBMK5SOMKVVWZRSA4RT1ZEVEEOKBY2VC0MO


Setting Foursquare get query

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting TORONTO venues Data

In [30]:
TORONTO_venues = getNearbyVenues(names=T_data['Neighborhood'],
                                   latitudes=T_data['Latitude'],
                                   longitudes=T_data['Longitude']
                                  )

The Beaches
The Danforth West
The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park
Deer Park
Rosedale
Cabbagetown
Church and Wellesley
Harbourfront
Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide
Harbourfront East
Design Exchange
Commerce Court
Roselawn
Forest Hill North
The Annex
Harbord
Chinatown
CN Tower
Stn A PO Boxes 25 The Esplanade
First Canadian Place
Christie
Dovercourt Village
Little Portugal
Brockton
High Park
Parkdale
Runnymede
Business Reply Mail Processing Centre 969 Eastern


In [31]:
print(TORONTO_venues.shape)
TORONTO_venues.head()

(834, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Grouping TORONTO venues

In [32]:
TORONTO_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
Brockton,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,15,15,15,15,15,15
Cabbagetown,30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30
Christie,15,15,15,15,15,15


Getting uniques categories

In [33]:
print('There are {} uniques categories.'.format(len(TORONTO_venues['Venue Category'].unique())))

There are 185 uniques categories.


one hot encoding

In [34]:
TORONTO_onehot = pd.get_dummies(TORONTO_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TORONTO_onehot['Neighborhood'] = TORONTO_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [TORONTO_onehot.columns[-1]] + list(TORONTO_onehot.columns[:-1])
TORONTO_onehot = TORONTO_onehot[fixed_columns]

TORONTO_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0

In [35]:
TORONTO_onehot.shape

(834, 185)

In [36]:
TORONTO_grouped = TORONTO_onehot.groupby('Neighborhood').mean().reset_index()
TORONTO_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.0

In [37]:
TORONTO_grouped.shape

(38, 185)

5 top venue

In [38]:
num_top_venues = 5

for hood in TORONTO_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = TORONTO_grouped[TORONTO_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0        Steakhouse  0.10
1             Hotel  0.07
2       Coffee Shop  0.07
3  Asian Restaurant  0.07
4              Café  0.07


----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2                Café  0.07
3      Farmers Market  0.07
4            Beer Bar  0.07


----Brockton----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4    Climbing Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1            Garden  0.06
2              Park  0.06
3  Recording Studio  0.06
4        Restaurant  0.06


----CN Tower----
             venue  freq
0   Airport Lounge  0.13
1  Airport Service  0.13
2    Boat or Ferry  0.07
3  Harbor / Marina  0.07
4      Coffee Shop  0.07


----Cabbagetown----
                venue  freq
0         Coffee Shop  0.10
1          

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

10 TOP VANUES STATS

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TORONTO_grouped['Neighborhood']

for ind in np.arange(TORONTO_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TORONTO_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Steakhouse,Coffee Shop,Café,Asian Restaurant,Hotel,Noodle House,Monument / Landmark,Colombian Restaurant,Pizza Place,Concert Hall
1,Berczy Park,Coffee Shop,Seafood Restaurant,Beer Bar,Café,Cocktail Bar,Farmers Market,Park,Museum,Comfort Food Restaurant,Concert Hall
2,Brockton,Coffee Shop,Café,Breakfast Spot,Grocery Store,Convenience Store,Restaurant,Burrito Place,Bar,Italian Restaurant,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Burrito Place,Recording Studio,Restaurant,Brewery,Light Rail Station,Skate Park
4,CN Tower,Airport Lounge,Airport Service,Plane,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Boutique,Harbor / Marina,Airport Terminal


K 5 CLUSTERING

In [41]:
# set number of clusters
kclusters = 5

TORONTO_grouped_clustering = TORONTO_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TORONTO_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 1, 2, 2, 2, 2, 1])

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TORONTO_merged = T_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TORONTO_merged = TORONTO_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TORONTO_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Wine Bar,Coworking Space,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
1,M4K,East Toronto,The Danforth West,43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Grocery Store,Spa,Pub,Pizza Place,Juice Bar,Indian Restaurant,Furniture / Home Store
2,M4L,East Toronto,The Beaches West,43.668999,-79.315572,1,Park,Sandwich Place,Pet Store,Fish & Chips Shop,Movie Theater,Pizza Place,Pub,Burrito Place,Burger Joint,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Gay Bar,Chinese Restaurant,Cheese Shop,Comfort Food Restaurant,Stationery Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Lawyer,Park,Bus Line,Swim School,Creperie,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TORONTO_merged['Latitude'], TORONTO_merged['Longitude'], TORONTO_merged['Neighborhood'], TORONTO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters